In [3]:
%pip install datasets

  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached aiohttp-3.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
  Using cached yarl-1.11.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)
  Using cached async_timeout

In [4]:
# data
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
amnesty_qa

The repository for explodinggradients/amnesty_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/explodinggradients/amnesty_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Repo card metadata block was not found. Setting CardData to empty.


Generating eval split: 0 examples [00:00, ? examples/s]

DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [5]:
amnesty_qa['eval']

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 20
})

Ok, so we need to create a HF dataset like this with our output data

In [31]:
import json
from datasets import Dataset

def create_dataset(gt_file = './data/rag_dataset.json', output_file='output_0.jsonl'):

    input_data = json.load(open(gt_file, 'r'))

    # Initialize lists to hold each field's data
    questions = []
    ground_truths = []
    answers = []
    contexts = []

    # Open and read the output.jsonl file
    with open(output_file, 'r') as outfile:
        for i, line in enumerate(outfile):
            # Load each line as a JSON object
            result = json.loads(line)

            # Retrieve relevant fields from the input_data and result
            question = input_data['examples'][i]['query']
            ground_truth = input_data['examples'][i]['reference_answer']
            answer = result['output']
            context = result['observations']  # Assuming 'observations' is used as 'contexts'

            # Append data to corresponding lists
            questions.append(question)
            ground_truths.append(ground_truth)
            answers.append(answer)
            contexts.append(context)

    # Create a dictionary to match the structure of the dataset
    dataset_dict = {
        'question': questions,
        'ground_truth': ground_truths,
        'answer': answers,
        'contexts': contexts
    }

    # Create the Huggingface dataset
    dataset = Dataset.from_dict(dataset_dict)

    return dataset


In [32]:
dataset = create_dataset(gt_file = './data/rag_dataset.json', output_file='output_0.jsonl')

In [28]:
# %pip install ragas langchain_aws

In [19]:
from ragas.metrics import (
    context_precision,
    faithfulness,
    context_recall,
    answer_correctness
)

# list of metrics we're going to use
metrics = [
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
]

In [26]:
from langchain.agents import AgentType,initialize_agent,load_tools
from langchain_aws import BedrockLLM, ChatBedrockConverse, ChatBedrock, BedrockEmbeddings
from langchain.tools import ShellTool
# from langchain_community.chat_models import BedrockChat

# llm = BedrockLLM(model_id="amazon.titan-text-premier-v1:0")

# llm = ChatBedrockConverse(model_id="anthropic.claude-3-sonnet-20240229-v1:0")

bedrock_llm = ChatBedrock(model_id = "us.anthropic.claude-3-sonnet-20240229-v1:0" )
bedrock_emb = BedrockEmbeddings(model_id = "amazon.titan-embed-text-v2:0" )

In [27]:
from ragas import evaluate
import nest_asyncio  # CHECK NOTES

# NOTES: Only used when running on a jupyter notebook, otherwise comment or remove this function.
nest_asyncio.apply()

result = evaluate(
    dataset,
    metrics=metrics,
    llm=bedrock_llm,
    embeddings=bedrock_emb
)


Evaluating:   0%|          | 0/232 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.


In [29]:
result

{'faithfulness': 0.8662, 'context_recall': 0.7527, 'context_precision': 0.3549, 'answer_correctness': 0.5808}

In [33]:
datasets = [create_dataset(gt_file = './data/rag_dataset.json', output_file=f'output_{i}.jsonl') for i in range(3)]

In [ ]:
for dataset in datasets:
    result = evaluate(
    dataset,
    metrics=metrics,
    llm=bedrock_llm,
    embeddings=bedrock_emb)
    
    print(result)


Evaluating:   0%|          | 0/232 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
